# 4.0 Computer Vision

In [ ]:
import pandas as pd
import numpy as np
import torch as tc
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

## 4.1 Getting our datsets

#### Please do not run the code below more than once

In [ ]:
train_data = datasets.FashionMNIST(
    root="DATA", train=True, download=True, transform=ToTensor(), target_transform=None
)

test_data = datasets.FashionMNIST(
    root="DATA", train=False, download=True, transform=ToTensor(), target_transform=None
)

#### Please do not run the code above more than once

In [ ]:
train_data
# seeing the first data
image,label = train_data[0]
image,label

In [ ]:
test_data

In [ ]:
classnames = train_data.classes
classnames

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

## 4.2 Visualising random sample data

In [ ]:
print(f"Image Shape: {image.shape}")
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
# gray scale
plt.imshow(image.squeeze(),cmap="gray")
plt.title(classnames[label])
plt.axis(False)

In [ ]:
# visualising random images
tc.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows,cols = 4,4
for i in range(1,rows*cols+1):
  random_idx = tc.randint(0,len(train_data),size=[1]).item()
  # print(random_idx)
  img,lab = train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap="gray")
  plt.title(classnames[lab])
  plt.axis(False)

### 4.2.1 Preparing Data Loader

In [ ]:
from torch.utils.data import DataLoader

# batch size hyper parameter
BATCH_SIZE = 32

# turn dataset into iterable
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

train_dataloader,test_dataloader

In [ ]:
print(f"DataLoaders: {train_dataloader,test_dataloader}")
print(f"Length of train data loader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of test data loader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
# checking training data loader
train_features_batch, train_label_batch = next(iter(train_dataloader))
train_features_batch.shape,train_label_batch.shape

In [ ]:
# show a sample
tc.manual_seed(42)
random_idx = tc.randint(0,len(train_features_batch),size=[1]).item()
img, lab = train_features_batch[random_idx],train_label_batch[random_idx]
plt.imshow(img.squeeze(),cmap="gray")
plt.title(classnames[lab])
plt.axis(False)
print(f"Image size: {img.shape}")
print(f"Label: {lab}, Label size: {lab.shape}")

## 4.3 Creating a model

In [ ]:
# building a baseline model
flatten_model = nn.Flatten()

# get a single sample
x = train_features_batch[0]
x.shape
# flatten x
output = flatten_model(x)
print(f"Shape before flattening: {x.shape}\nShape after flattening: {output.shape}")

In [ ]:
class FashionMNISTModelV0(nn.Module):
  def __init__(self,input_shape:int,hidden_unit:int,output_shpe:int):
    super().__init__()
    self.layer_stack = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=input_shape,out_features=hidden_unit),
      nn.Linear(in_features=hidden_unit,out_features=output_shpe)
    )
    
  def forward(self,x):
    return self.layer_stack(x)

In [ ]:
tc.manual_seed(42)
model0 = FashionMNISTModelV0(input_shape=784,hidden_unit=10,output_shpe=len(classnames))

model0

### 4.3.1 Setup loss function and optimiser

In [20]:
# import accuracy functions
from helper import accuracy_fn

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimiser = tc.optim.SGD(params=model0.parameters(),lr=0.1)

In [26]:
# setting up timer function
from timeit import default_timer as timer


def print_train_time(start: float, end: float, devive: tc.device = None):
    """Prints difference between start time and end time"""
    total_time = end - start
    print(f"Train time on {devive}: {total_time:.3f} seconds")
    return total_time